In [4]:
import wandb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

"""
https://stackoverflow.com/questions/21654635/scatter-plots-in-pandas-pyplot-how-to-plot-by-category
"""

def afterPoint(str):
    return str.split('.')[-1]



tag = "active_learning_sbmal_v2"

api = wandb.Api(timeout=200)
runs = api.runs("tum_daml_ba_antoniooroz/Guided Research", {"tags" : tag})

runs_per_starting_class = {}

for run in runs:
    if run.config["debug"] or run.config["model/type"]!='ModelType.MCD_GCN':
        continue
    
    
    
    starting_class = str(run.config["experiment/active_learning_starting_class"])
    
    if starting_class in runs_per_starting_class:
        runs_per_starting_class[starting_class].append(run)
    else:
        runs_per_starting_class[starting_class] = [run]
        
for key, runs in runs_per_starting_class.items():
    accuracies = []
    deviations = []
    models = []
    selectors = []
    
    keys = {
        'L2_DISTANCE': 300,
        'UNCERTAINTY': 100,
        'RANDOM': 200
    }
    
    keys2 = {
        'ALEATORIC': 10,
        'EPISTEMIC': 20
    }
    
    keys3 = {
        'ISOLATED': 1,
        'PROPAGATED': 2
    }
    
    runs.sort(key=lambda x: 
        keys[afterPoint(x.config["experiment/active_learning_selector"])] + keys2[afterPoint(x.config['experiment/active_learning_selector_uncertainty_mode'])] + keys3[afterPoint(x.config['experiment/active_learning_selector_network_mode'])]
    )
    
    for run in runs:
        accuracy = run.summary["mean/eval/VALTEST/PROPAGATED/accuracy"]*100
        deviation = run.summary["std/eval/VALTEST/PROPAGATED/accuracy"]*100
        
        model = afterPoint(run.config["model/type"])
        
        if run.config['experiment/active_learning_l2_distance_use_centroids']==True and afterPoint(run.config['experiment/active_learning_selector'])=='L2_DISTANCE':
            continue
    
        active_learning_selector = afterPoint(run.config["experiment/active_learning_selector"])
        if active_learning_selector == "UNCERTAINTY":
            uncertainty_name = "ALEA" if afterPoint(run.config['experiment/active_learning_selector_uncertainty_mode']) == 'ALEATORIC' else 'EPIST'
            prop_name = 'ISO' if afterPoint(run.config['experiment/active_learning_selector_network_mode']) == 'ISOLATED' else 'PROP'
            active_learning_selector = f"{uncertainty_name} {prop_name}"

        accuracies.append(accuracy)
        deviations.append(deviation)
        models.append(model)
        selectors.append(active_learning_selector)
        
    df = pd.DataFrame(dict(x=selectors, y=accuracies, err=deviations, label=models))
    groups = df.groupby('label')
    
    if str(key)!='[0]':
        continue
    
    fig, ax = plt.subplots(figsize=(7, 3), dpi=1200)
    ax.margins(0.05)
    for name, group in groups:
        ax.set_ylim([0.0, 0.8])
        ax.errorbar(group.x, group.y, yerr=group.err, fmt='.k', ms=12, color='#003459', ecolor='#003459', label=name)
        ax.yaxis.set_ticks(np.arange(0.0,85.0, 10))
    ax.legend(numpoints=1)
    print(key)
    plt.title('MCD GCN')
    plt.show()


wandb: WARNING Run.username is deprecated. Please use Run.entity instead.
wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=9 sec). Create a new API with an integer timeout larger than 9, e.g., `api = wandb.Api(timeout=19)` to increase the graphql timeout.
wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=9 sec). Create a new API with an integer timeout larger than 9, e.g., `api = wandb.Api(timeout=19)` to increase the graphql timeout.


KeyError: 'FIXED'